In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from wetterdienst.provider.dwd.observation import DwdObservationRequest, DwdObservationDataset, DwdObservationPeriod, DwdObservationResolution
from wetterdienst import Wetterdienst, Resolution, Period, Parameter
from pprint import pprint

In [44]:
from wetterdienst import Settings
Settings.tidy = True
Settings.humanize =True
Settings.si_units = True
API = Wetterdienst(provider="dwd", network="observation")

def dwd_request(params, start, end, station_ids):
    """Retrieve stations_result of DWD that measure temperature."""
    request = DwdObservationRequest(
        parameter=[params],
        resolution=DwdObservationResolution.HOURLY,
        start_date=start,
        end_date=end,
        period=DwdObservationPeriod.HISTORICAL,
    )
    print(dir(request))
    print(request.discover())
    pprint(request.describe_fields(
            dataset=request.,
            resolution=DwdObservationResolution.DAILY,
            period=DwdObservationPeriod.RECENT,
            language="de"))
    
    # stations = request.filter_by_distance(latitude=50.0, longitude=8.9, distance=100)
    stations = request.filter_by_station_id(station_ids)
    

    #print(df.head())
    return stations#["station_id"]


In [45]:
start_date="2021-01-01 00:00:00+00:00"
end_date="2021-12-31 23:59:00+00:00"
parameters = ["wind_speed", "sunshine_duration"]
station_ids=["07341", "03631"]#, "01420"]
#station_ids=[]

date=pd.date_range(start_date, end_date, freq="H")
# stations = dwd_request(parameters[0], start_date, end_date, station_ids[0])
# df=stations.values.all().df
# date=df.date

header = pd.MultiIndex.from_product([station_ids, parameters], names=['station_id', 'parameter'])
total_df=pd.DataFrame(columns=header)
total_df.loc[:,("date", "date")]=date
total_df.loc[:,("date", "timestamp")]=date.astype('int64')//10**9

for parameter in parameters:
    for id in station_ids:
        print("Receiving Parameter ", parameter, " from station ", id)
        station = dwd_request(parameter, start_date, end_date, id)
        df = station.values.all().df
        # print(df.value.head())
        total_df.loc[id, parameter]=df.value

#total_df.head(30)

start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]


Receiving Parameter  wind_speed  from station  07341
['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_all', '_base_columns', '_coerce_meta_fields', '_data_range', '_dataset_accessor', '_dataset_base', '_format_unit', '_get_periods', '_has_datasets', '_has_tidy_data', '_historical_interval', '_interval', '_now_interval', '_now_local', '_parameter_base', '_parameter_to_dataset_mapping', '_parse_dataset_and_parameter', '_parse_parameter', '_parse_parameter_and_dataset', '_parse_period', '_parse_station_id', '_period_base', '_period_type', '_recent_interval', '_resolution_base', '_resolution_type', '_setup_discover_filter', '_tz', '_unique_dataset', '_unit_tree'

AttributeError: 'DwdObservationRequest' object has no attribute '_dataset_tree'

In [37]:
path="..\data\Weatherdata_2021.csv"

In [38]:
total_df.to_csv(path)

In [39]:
new_df=pd.read_csv(path, header=[0, 1], index_col=0)
new_df[:150]

station_id      07341                        03631                    \
parameter  wind_speed sunshine_duration wind_speed sunshine_duration   
0                 NaN               NaN        NaN               NaN   
1                 NaN               NaN        NaN               NaN   
2                 NaN               NaN        NaN               NaN   
3                 NaN               NaN        NaN               NaN   
4                 NaN               NaN        NaN               NaN   
..                ...               ...        ...               ...   
145               NaN               NaN        NaN               NaN   
146               NaN               NaN        NaN               NaN   
147               NaN               NaN        NaN               NaN   
148               NaN               NaN        NaN               NaN   
149               NaN               NaN        NaN               NaN   

station_id                       date                       wind_speed  \
parameter                        date     timestamp Unnamed: 7_level_1   
0           2021-01-01 00:00:00+00:00  1.609459e+09                NaN   
1           2021-01-01 01:00:00+00:00  1.609463e+09                NaN   
2           2021-01-01 02:00:00+00:00  1.609466e+09                NaN   
3           2021-01-01 03:00:00+00:00  1.609470e+09                NaN   
4           2021-01-01 04:00:00+00:00  1.609474e+09                NaN   
..                                ...           ...                ...   
145         2021-01-07 01:00:00+00:00  1.609981e+09                NaN   
146         2021-01-07 02:00:00+00:00  1.609985e+09                NaN   
147         2021-01-07 03:00:00+00:00  1.609988e+09                NaN   
148         2021-01-07 04:00:00+00:00  1.609992e+09                NaN   
149         2021-01-07 05:00:00+00:00  1.609996e+09                NaN   

station_id  sunshine_duration  
parameter  Unnamed: 8_level_1  
0                         NaN  
1                         NaN  
2                         NaN  
3                         NaN  
4                         NaN  
..                        ...  
145                       NaN  
146                       NaN  
147                       NaN  
148                       NaN  
149                       NaN  

[150 rows x 8 columns]